# Leaf Classfication 
A For CZ4041 Machine Learning Assignment from PT3 in AY2018/2019 Semester 2.
The group members are:
- LIU Yiqing
- LUO Bingyi
- TENG He Xu
- WANG Jia
- YI Zhiyue
- ZHAO Ziru

## Import necessary libraries and Define Constants

In [27]:
import os
import csv
import pandas as pd
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.utils import to_categorical
from keras.callbacks import History 
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit

history1 = History()
history2 = History()
history3 = History()

LABEL_PATH = 'data/'
TRAIN_FILE_NAME = 'train.csv'
TEST_FILE_NAME = 'test.csv'


## Load From CSV

Load features and labels from the train csv file

In [119]:
train_data_frame = pd.read_csv(LABEL_PATH + TRAIN_FILE_NAME)

train_data_frame = train_data_frame.drop(['id'], axis=1)

y = train_data_frame.pop('species')
classes = np.unique(y)

y = to_categorical(LabelEncoder().fit(y).transform(y))

x = StandardScaler().fit(train_data_frame).transform(train_data_frame)
#x = train_data_frame.get_values()


In [120]:
sss = StratifiedShuffleSplit(n_splits=10, test_size=0.2,random_state=12345)

train_index, validation_index = next(iter(sss.split(x, y)))
train_x, validate_x = x[train_index], x[validation_index]
train_y, validate_y = y[train_index], y[validation_index]
print("train_x dimention: ",train_x.shape)
print("validate_x dimention:   ",validate_x.shape)

train_x dimention:  (792, 192)
validate_x dimention:    (198, 192)


In [121]:
no_of_classes = len(np.unique(train_y, axis=0))

In [122]:
train_x

array([[ 1.69134715,  2.13184307, -0.6333894 , ..., -0.52001246,
         0.62307994, -0.85338743],
       [ 0.00839582, -0.48340807,  0.19823493, ..., -0.09453821,
        -0.65214311, -0.85338743],
       [-0.58560191, -0.53372315,  1.63472622, ..., -0.52001246,
        -0.65214311, -0.29551715],
       ..., 
       [ 0.70138472,  0.72361309, -0.10415984, ..., -0.31980737,
        -0.65214311,  1.07773579],
       [ 0.20637817,  1.98094933, -0.48219201, ..., -0.52001246,
        -0.65214311, -0.72463124],
       [-0.48661074, -0.08106768,  1.71032491, ..., -0.52001246,
        -0.22709294, -0.85338743]])

## Build model

Use ensemble learning method to predict the value

In [123]:
model1 = Sequential()

model1.add(Dense(250, activation='relu', input_dim = train_x.shape[1]))
model1.add(Dropout(0.2))
model1.add(Dense(150, activation='relu'))
model1.add(Dropout(0.4))
model1.add(Dense(no_of_classes, activation=tf.nn.softmax))

In [124]:
model2 = Sequential()

model2.add(Dense(1000, activation='tanh', input_dim = train_x.shape[1]))
model2.add(Dense(1000, activation='relu'))
model2.add(Dense(1000, activation='relu'))
model2.add(Dense(no_of_classes, activation=tf.nn.softmax))

In [125]:
model3 = Sequential()

model3.add(Dense(500, activation='relu', input_dim = train_x.shape[1]))
model3.add(Dropout(0.4))
model3.add(Dense(500, activation='relu'))
model3.add(Dropout(0.2))
model3.add(Dense(no_of_classes, activation=tf.nn.softmax))

In [126]:
model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_23 (Dense)             (None, 250)               48250     
_________________________________________________________________
dropout_13 (Dropout)         (None, 250)               0         
_________________________________________________________________
dense_24 (Dense)             (None, 150)               37650     
_________________________________________________________________
dropout_14 (Dropout)         (None, 150)               0         
_________________________________________________________________
dense_25 (Dense)             (None, 99)                14949     
Total params: 100,849
Trainable params: 100,849
Non-trainable params: 0
_________________________________________________________________


In [127]:
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_26 (Dense)             (None, 1000)              193000    
_________________________________________________________________
dense_27 (Dense)             (None, 1000)              1001000   
_________________________________________________________________
dense_28 (Dense)             (None, 1000)              1001000   
_________________________________________________________________
dense_29 (Dense)             (None, 99)                99099     
Total params: 2,294,099
Trainable params: 2,294,099
Non-trainable params: 0
_________________________________________________________________


In [128]:
model3.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_30 (Dense)             (None, 500)               96500     
_________________________________________________________________
dropout_15 (Dropout)         (None, 500)               0         
_________________________________________________________________
dense_31 (Dense)             (None, 500)               250500    
_________________________________________________________________
dropout_16 (Dropout)         (None, 500)               0         
_________________________________________________________________
dense_32 (Dense)             (None, 99)                49599     
Total params: 396,599
Trainable params: 396,599
Non-trainable params: 0
_________________________________________________________________


In [129]:
model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model1.fit(train_x, train_y, epochs = 50, verbose=0, validation_data=(validate_x, validate_y), callbacks=[history1])

In [130]:
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model2.fit(train_x, train_y, epochs = 10, verbose=0, validation_data=(validate_x, validate_y), callbacks=[history2])

In [131]:
model3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model3.fit(train_x, train_y, epochs = 20, verbose=0, validation_data=(validate_x, validate_y), callbacks=[history3])

## Save Models

In [143]:
model1.save('models/model_1_0.52331.h5')
model2.save('models/model_2_0.52331.h5')
model3.save('models/model_3_0.52331.h5')

## Test

In [132]:
test_data_frame = pd.read_csv(LABEL_PATH + TEST_FILE_NAME)

index = test_data_frame.pop('id')

test_x = StandardScaler().fit(test_data_frame).transform(test_data_frame)
#test_x = test_data_frame.get_values()

In [133]:
test_y_1 = model1.predict_classes(test_x)
test_y_2 = model2.predict_classes(test_x)
test_y_3 = model3.predict_classes(test_x)

In [134]:
test_y = [
    {
        'name': 'test_y_1',
        'predict': test_y_1,
        'loss': history1.history['loss'][-1]
    }, {
        'name': 'test_y_2',
        'predict': test_y_2,
        'loss': history2.history['loss'][-1]
    }, {
        'name': 'test_y_3',
        'predict': test_y_3,
        'loss': history3.history['loss'][-1]
    }, 
]

test_y

[{'loss': 0.022677927023985169,
  'name': 'test_y_1',
  'predict': array([51, 50,  1, 19, 14,  3,  3, 28, 84,  8, 43, 74, 75, 10, 52, 46, 45,
         73, 13, 71, 61, 68, 57, 77,  1, 70, 28, 15, 35, 70, 53, 74, 47, 50,
          4, 36, 14, 55, 36, 93,  8, 32,  8,  9, 71, 70, 38, 23, 94, 18, 17,
          5, 55, 94, 14, 86, 62, 33, 51, 98, 88, 56, 21, 59, 65, 11, 48,  5,
         13,  4, 54, 57, 29,  7, 31, 98, 92, 84, 25, 10, 61, 43, 85, 24,  1,
          2, 23, 83, 40, 22, 48, 90, 25, 21, 37, 56, 41, 95,  7, 89, 98, 77,
          3, 12, 31, 84, 53, 96, 64, 72, 93, 93, 67, 30,  8, 88, 60, 87,  6,
         57, 34, 90, 60, 17, 75, 27, 51, 31, 39, 23, 24,  2, 41, 61, 24, 97,
         29, 28, 68, 81, 42, 51, 86, 62, 60, 52, 95, 64, 42, 96, 95, 20, 59,
         35, 86,  1, 56, 38, 43, 75, 20, 60, 46, 79, 22, 79, 69, 87, 65, 97,
         75, 21, 29, 21, 11, 10, 58, 94, 27, 22, 15, 45, 89, 54, 43,  5, 23,
         94, 40, 49, 89, 72, 36, 11, 90, 95, 18, 91, 29, 64, 80,  6, 78, 45,
         28

In [135]:
data_grid = np.zeros((len(test_y_1), len(classes)))

for i in range(len(test_y_1)):
    result = set([test_y_1[i], test_y_2[i], test_y_3[i]])
    print(len(result))
    if len(result) == 1:
        data_grid[i][test_y_1[i]] = 1
    elif len(result) == 2: 
        if test_y_1[i] == test_y_2[i] or test_y_1[i] == test_y_3[i]:
            data_grid[i][test_y_1[i]] = 1
        else:
            data_grid[i][test_y_2[i]] = 1
    else:
        minLoss = min([history1.history['loss'][-1], history2.history['loss'][-1], history3.history['loss'][-1]])
        if(history1.history['loss'][-1] == minLoss): 
            data_grid[i][test_y_1[i]] = 1
        elif (history2.history['loss'][-1] == minLoss): 
            data_grid[i][test_y_2[i]] = 1
        else:
            data_grid[i][test_y_3[i]] = 1

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
3
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
2
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
3
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
2
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [136]:
test_y_2[61]

56

In [137]:
prediction = pd.DataFrame(data_grid, index = index, columns = classes)

In [138]:
np.shape(test_y_1)

(594,)

In [139]:
with open('submission.csv','w') as file:
    file.write(prediction.to_csv())
